# Oracle AutoML practical approach

## The following code can be used in an Oracle Machine Learning Notebook (Oracle Cloud Infrastructure).

Divide data into two subsets: train dataset and test dataset.

In [ ]:
%script
CREATE TABLE MOVIESTREAM_CHURN_TRAIN AS SELECT * FROM MOVIESTREAM_CHURN SAMPLE (85) SEED (30); 
CREATE TABLE MOVIESTREAM_CHURN_TEST AS SELECT * FROM MOVIESTREAM_CHURN MINUS SELECT * from MOVIESTREAM_CHURN_TRAIN;

Importing libraries

In [2]:
%python
import pandas as pd
import oml
from oml import automl

Prepare the training dataset: the __sync__ function returns in the notebook session an __oml.DataFrame__ object which is a proxy pointing to the database object (table, view or query).

In [3]:
%python
MCT = oml.sync("MOVIESTREAM", table = "MOVIESTREAM_CHURN_TRAIN")
MCT_X_cl, MCT_y_cl = MCT.drop('IS_CHURNER'), MCT['IS_CHURNER']

__Automatic Algorithm Selection__. 
_Score metric_ parameter represents the evaluation metric used to evaluate the model performance. There are several metrics that can be used for binary classification: _accuracy_ (default), _precision_, _precision_weighted_, _precision_macro_, _precision_micro_, _recall_, _recall_weighted_, _recall_macro_, _recall_micro_, _f1_, _f1_macro_, _f1_micro_, _f1_weighted_, _roc_auc_. The degree of parallelism is determined by the _parallel_ parameter. The second step (_select_) uses this setup and rank the available proxy models to find the “best models”. The parameter “$k$” specifies the number of top models to be included in the return list. Models returned: 'rf' = Random Forest (RF) – the best model, 'dt' = Decision Tree (DT), 'svm_gaussian' = Support Vector Machine with Gaussian kernel (SVMG) and 'glm_ridge' = Generalized Linear Model for Regression (GLMR).

In [5]:
%python
as_mct_cl = automl.AlgorithmSelection(mining_function='classification', score_metric='accuracy', parallel=2)
mct_alg_ranking_cl = as_mct_cl.select(MCT_X_cl, MCT_y_cl, k=4)
print("Ranked algorithms:\n", mct_alg_ranking_cl)
best_mct_alg_cl = next(iter(dict(mct_alg_ranking_cl).keys()))
print("Best algorithm: ", best_mct_alg_cl)

__Automatic Feature Selection__. _FeatureSelection_ configures this pipeline stage for _classification_, using _accuracy_ as the evaluation metric and sets the degree of parallelism for evaluating the subsets of features at value $2$. The _reduce_ function correlates data with the model ('rf') and returns the reduced set of features/attributes/columns, optimized to improve the model performance in relation to the evaluation or score metric.

In [6]:
%python
fs_mct_cl = automl.FeatureSelection(mining_function = 'classification', score_metric = 'accuracy', parallel=2)
selected_mct_features_cl = fs_mct_cl.reduce('dt', MCT_X_cl, MCT_y_cl)
MCT_X_reduced_cl = MCT_X_cl[:,selected_mct_features_cl]
print("Selected columns:", MCT_X_reduced_cl.columns)

Reducing the number of columns in the training/test data sets (subsets of features)

In [17]:
%sql
create table MOVIESTREAM_CHURN_RED_TRAIN as 
select CUST_ID, IS_CHURNER,
       AGE, CREDIT_BALANCE, EDUCATION, GENDER, WORK_EXPERIENCE, YRS_CURRENT_EMPLOYER, YRS_CUSTOMER, APP_MOBILE, AVG_DISC_M12, 
       AVG_DISC_M12_14, AVG_DISC_M3_8, AVG_DISC_M6, AVG_DISC_M6_8, AVG_NTRANS_M3_11, AVG_NTRANS_M3_14, AVG_NTRANS_M3_5, AVG_NTRANS_M3_8
from MOVIESTREAM_CHURN_TRAIN

create table MOVIESTREAM_CHURN_RED_TEST as 
select CUST_ID, IS_CHURNER,
       AGE, CREDIT_BALANCE, EDUCATION, GENDER, WORK_EXPERIENCE, YRS_CURRENT_EMPLOYER, YRS_CUSTOMER, APP_MOBILE, AVG_DISC_M12, 
       AVG_DISC_M12_14, AVG_DISC_M3_8, AVG_DISC_M6, AVG_DISC_M6_8, AVG_NTRANS_M3_11, AVG_NTRANS_M3_14, AVG_NTRANS_M3_5, AVG_NTRANS_M3_8
from MOVIESTREAM_CHURN_TEST

__Automatic Model Tuning__ - takes the details generated from the previous steps and use this to generate a tuned model. A time limit of five minutes has been set.

In [ ]:
%python
mt_mct_cl = automl.ModelTuning(mining_function = 'classification', parallel=2)
results_cl = mt_mct_cl.tune('rf', MCT_X_reduced_cl, MCT_y_cl, time_budget = 300)
tuned_model_cl = results_cl['best_model']

__Save tuned model in the database__.

In [20]:
%python
dt_export = tuned_model_cl.export_sermodel(table='model_rf_moviestream_churn')

__Restore tuned model from the database__.

In [22]:
%script
DECLARE
  v_blob blob;
    
BEGIN
  dbms_lob.createtemporary(v_blob, FALSE);
  SELECT SERVAL INTO v_blob FROM "model_rf_moviestream_churn";
  dbms_data_mining.import_sermodel(v_blob, 'model_rf_moviestream_churn');
  dbms_lob.freetemporary(v_blob);
END;             

__Score the tuned machine learning model via SQL functions__. The SQL functions (PREDICTION, PREDICTION_PROBABILITY) can apply a machine learning model schema object to the data.

In [23]:
%script
create table RF_Train_Prediction
as 
SELECT CUST_ID as ID, IS_CHURNER as ACTUALVALUE,
       PREDICTION(model_rf_moviestream_churn USING *) AS PREDICTEDVALUE,
       round(PREDICTION_PROBABILITY(model_rf_moviestream_churn USING *),2) AS PREDICTIONCONFIDENCE
FROM MOVIESTREAM_CHURN_RED_TRAIN;